In [10]:
# importeren libs
import numpy as np
import pandas as pd
import pickle
import copy
import itertools

import Importeer 
import Hulpfuncties
import CrewCombined
import Dataframe
import Controleren
from Class import MogelijkeCombinaties, Rooster

In [3]:
filehandler = open('dfSprint1_eind.obj', 'rb') 
dfSprint = pickle.load(filehandler)
filehandler = open('dfKalenderCrew1.obj', 'rb') 
dfKalenderCrew = pickle.load(filehandler)
filehandler = open('dfKalenderRoom1.obj', 'rb') 
dfKalenderRoom = pickle.load(filehandler)
filehandler = open('dfSprintAddprep1.obj', 'rb') 
dfSprintAddprep = pickle.load(filehandler)
filehandler = open('dfSprintAddpost1.obj', 'rb') 
dfSprintAddpost = pickle.load(filehandler)
filehandler = open('dfCrew1.obj', 'rb') 
dfCrew = pickle.load(filehandler)

In [4]:
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name='Additional tasks')
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfAdditionalTasks = dfAdditionalTasks[dfAdditionalTasks['Task'] != 0].reset_index(drop=True)
dfAdditionalTasks['aantal keer'].replace(0, 1, inplace=True)
dfSprintAdd = pd.DataFrame((dfAdditionalTasks.loc[~dfAdditionalTasks['Task'].str.contains('S1')]) ).reset_index(drop=True)
dfSprintAdd = pd.DataFrame((dfSprintAdd[dfSprintAdd['Sprint']==1]) ).reset_index(drop=True)
dfSprintAdd['Voltooid'] = False
dfSprintAdd.loc[:, 'Crew'] = [20] * len(dfSprintAdd)
dfSprintAdd['Crew'] = dfSprintAdd['Crew'].astype(object)

In [ ]:
dfSprintAdd.iloc[:,[0,20,21,22,27,30,31]]

In [ ]:
som = pd.DataFrame(data = [range(0,13), [0]*13]).T
som.columns=['Crew', 'som']

In [ ]:
som

Dennis is de crewdirector.

In [ ]:
sum = 0
for i in range(0,10*5):
      sum += dfKalenderCrew.iloc[i,:].DagRooster.dfRooster.iloc[0,1:].sum()

In [ ]:
sum

In [ ]:
for i in range(0,1*5):
    for j in range(0,13):
        som.som.at[j] += float(dfKalenderCrew.iloc[i,:].DagRooster.dfRooster.iloc[0,j+1])
        

In [ ]:
som.som.sum()

In [ ]:
dfSprint.iloc[:,[0,20,21,22,23,25,27,28,29,30,31]]

In [ ]:
som[som.som>40]

combinaties 8 of 12 inzitten

In [ ]:
import itertools

lst = som[som.som>40].Crew.tolist()
combs = []

for i in range(1, len(lst)+1):
    els = [list(x) for x in itertools.combinations(lst, i)]
    combs.extend(els)

In [ ]:
combs

In [ ]:
a= [lijst for lijst in combs if len(lijst) == 3]
b= [lijst for lijst in a if (12 in lijst) or (8 in lijst)]

In [ ]:
def InplannenAdddubbel(taskdf, dfKalenderCrew, dfKalenderRoom, dfAdd, crewlijst, inplannen = False): 

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempAdd = pickle.loads(pickle.dumps(dfAdd))
    gelukt = False

    dagIndex = 0
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    maxDagIndex = 50
    
    while halfuur < 40*2:
        for i in crewlijst:
            if (i == 20):
                break
            print(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,crewlijst[0]+1] , dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,crewlijst[1]+1] ,dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,crewlijst[2]+1] )
            if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)):
                print("controle wordt op false gezet, dagindex: " + str(dagIndexKopie) + " " + str(i))
                controle = False
                break 
            lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
#         print(lijstVrijeUren)
#         print("check: " + str((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle))
        if ((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle):
            rijbegin , rijeind = Hulpfuncties.contains([0], lijstVrijeUren)




            if (controle == True):
                print("volgende stap")
                for i in crewlijst:
                    if i == 20:
                        break
#                         print("halfuur: " + str(halfuur) + "crew: " + str(i) + "dagIndex: " + str(dagIndexKopie))

                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task 
                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5)

#                     print(taskdf)
                    dfAdd.at[dfAdd.index[dfAdd['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                    halfuur +=1
                    if (halfuur == 40*2):
                        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
                        print(str(taskdf.Task) + "is daadwerkelijk ingepland")

                        if inplannen:
                            return [True, dfKalenderCrew, dfKalenderRoom, dfAdd]
                        else:
                            return [True, dfTempCrew, dfTempRoom, dfTempAdd]

        else:

            dagIndexKopie+=1

            controle = True
            lijstVrijeUren = [0]*16
            if (dagIndexKopie == maxDagIndex):
                break

    print(str(taskdf.Task) + " Niet ingepland of: " + str(gelukt))
    return [gelukt, dfTempCrew, dfTempRoom, dfTempAdd]

In [ ]:
c,d,e,f = InplannenAdd(dfSprintAdd.iloc[8,:], dfKalenderCrew, dfKalenderRoom, dfSprintAdd, b[0])

In [ ]:
dfKalenderCrew.iloc[0,:].DagRooster.dfRooster.iloc[0,3+1]

In [ ]:
for i in b:
    c,d,e,f = InplannenAdd(dfSprintAdd.iloc[8,:], dfKalenderCrew, dfKalenderRoom, dfSprintAdd, i)
    if c:
        print("combinatie gevonden " + str(i))
        break

In [ ]:
dfKalenderCrew.iloc[5,:].DagRooster.dfRooster.iloc[0,0:4]

In [2]:
crew = Importeer.Crew()

In [3]:
crew

,Crew members,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6,Skill 7,Skill 8,Skill 9,...,Skill 11,Skill 12,Skill 13,mon,tue,wed,thu,fri,Additional,Uren
0,Alyssa,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,1,1,0,280
1,Chris,1,0,0,0,0,1,1,1,1,...,0,1,0,0,1,0,1,0,0,140
2,Dennis,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,0,350
3,Iris,1,0,0,0,0,1,0,1,0,...,1,0,1,1,1,0,1,0,0,210
4,Jurgen,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,1,1,0,0,210
5,Lois,1,1,1,0,0,0,1,0,1,...,0,0,0,1,1,1,1,1,must have acquired skill 8 by the end of sprint 4,350
6,Melanie,1,0,1,0,0,0,0,1,1,...,1,0,0,1,1,0,1,1,0,280
7,Mitch,1,0,1,0,0,0,0,1,1,...,1,1,1,1,1,0,1,1,0,280
8,Rudy,1,1,0,0,0,0,1,1,1,...,1,0,0,1,1,1,1,1,0,350
9,Shane,1,0,0,1,0,0,0,1,1,...,0,1,1,1,1,1,1,0,0,280


In [5]:
def CheckCrewUren(crewlijst, dfCrew, duration):
    
    tempCrew = pickle.loads(pickle.dumps(dfCrew))
    for crew in crewlijst:
        if crew == 20:
            return [True, dfCrew]
        elif (dfCrew['Uren'].iloc[crew] >= duration):
            dfCrew['Uren'].iloc[crew] -= duration
        else:
            return [False, tempCrew]
    return [True, dfCrew]

In [6]:
def InplannenAdd(taskdf, dfKalenderCrew, dfKalenderRoom, dfSprint, dfAdd, dfCrew, prep = True, inplannen = False): 

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempAdd = pickle.loads(pickle.dumps(dfAdd))
    dfCopyCrew = pickle.loads(pickle.dumps(dfCrew))
    
    gelukt = False

    if prep:
        dagIndex = 0
        weken = [1]
        maxDagIndex = 5
    else:
        dagIndex = 40
        weken = [9,10]
        maxDagIndex = 50
    # crew zelf maken
    crew = []

    if (taskdf.AdditionalReq == 0):
        
        task = taskdf.Task[5:]
        crewlijst = dfSprint[dfSprint.Task == task].iloc[0,:].Crew
        minCrew = taskdf['People.1']

        crewlijst = Hulpfuncties.somOverigeUren(crewlijst, weken, dfKalenderCrew)
        crewList = crewlijst['crew'].tolist()

        for crew in crewList: # checken voor het totaal aantal uren
            if (dfCrew['Uren'].iloc[crew] < taskdf["DurationExp.1"]):
                crewlijst = crewlijst[crewlijst.crew != crew]
        
        crewlijst = crewlijst['crew'].iloc[:minCrew].to_list()
        crewlijst.extend([20]*(6-minCrew))
       
    elif ((len(taskdf.AdditionalReq) > 5) & (len(taskdf.AdditionalReq) < 8)):
        try:
            crewlijst = dfSprint[dfSprint.Task == taskdf.AdditionalReq].iloc[0,:].Crew
        except:
            filehandler = open('dfSprint3_eind.obj', 'rb') # Moet worden verandert met de sprint
            dfSprintVorig = pickle.load(filehandler)
            crewlijst = dfSprintVorig[dfSprintVorig.Task == taskdf.AdditionalReq].iloc[0,:].Crew
            
    elif (len(taskdf.AdditionalReq) > 8 ):
        taskA = taskdf.AdditionalReq[0:7]
        taskB = taskdf.AdditionalReq[-7:]
        taskACrew = dfSprint[dfSprint.Task == taskA].iloc[0,:].Crew
        taskBCrew = dfSprint[dfSprint.Task == taskB].iloc[0,:].Crew
        crewlijst = [20]*6
        b = list(set(taskACrew)&set(taskBCrew))
        try:
            b.remove(20)
        except:
            None
        crewlijst[0] = b[0]
        
    a = list(set(crewlijst))
    try:
        a.remove(20)
    except:
        None
    aantalCrew = len(a)
    kolom = 1
    if (int(taskdf['People.1'])==int(aantalCrew)):
        kolom = 1
    elif (int(taskdf['People.2'])==int(aantalCrew)):
        kolom = 2
    elif (int(taskdf['People.3'])==int(aantalCrew)):
        kolom = 3
    
 # tot en met hier klopt de functie
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    
    if (int(round(taskdf[str("DurationExp." + str(kolom))]*2)) == 0):
        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
        gelukt = True
    
    else:
        
            
        while halfuur < int(round(taskdf[str("DurationExp." + str(kolom))]*2)):
            for i in crewlijst:
                if (i == 20):
                    break
#                 print("de controle is: " + str(((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5))))
                if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)):
                    controle = False
                    break 
                lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment

            if ((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle):
                rijbegin , rijeind = Hulpfuncties.contains([0], lijstVrijeUren)

                kamerGevonden = False

                kamers = Hulpfuncties.ZoekKamers(taskdf.iloc[14:20], dfAdd)

                ## hier nog even naar kijken of het klopt dat er niet naar rijeind en begin wordt gekeken !TODO
                for kamer in kamers:
                    if (Hulpfuncties.contains([0], dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[1:].tolist()) != False):

                        if inplannen:
                            dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1] = taskdf.Task
                        kamerGevonden = True
                        controle = True
                        break
                    elif ((kamer == kamers[-1]) & (kamerGevonden == False)):                       
                        controle = False

                if (controle == True):
                    for i in crewlijst:
                        if i == 20:
                            break
#                         print("halfuur: " + str(halfuur) + "crew: " + str(i) + "dagIndex: " + str(dagIndexKopie))
                        
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task 
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5)

    #                     print(taskdf)
                        dfAdd.at[dfAdd.index[dfAdd['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                        halfuur +=1

                        if (halfuur == (int(round(taskdf[str("DurationExp." + str(kolom))]*2)))):
                            
                            geluktCrew, dfCrew = CheckCrewUren(crewlijst, dfCrew, (int(round(taskdf[str("DurationExp." + str(kolom))]))))
                            if not geluktCrew:
                                print("er is iets fout gegaan")
                            dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
                            print(str(taskdf.Task) + "is daadwerkelijk ingepland")
                            
                            if inplannen:
                                return [True, dfKalenderCrew, dfKalenderRoom, dfAdd, dfCrew]
                            else:
                                return [True, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]

            else:

                dagIndexKopie+=1

                controle = True
                lijstVrijeUren = [0]*16
                if (dagIndexKopie == maxDagIndex):
                    break

    print(str(taskdf.Task) + " Niet ingepland of: " + str(gelukt))
    return [gelukt, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]

In [21]:
def InplannenOpdracht(taskdf, dagIndex, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfAddPost, dfSprint, dfCrew, crewA = []): 
# task moet een rij van het dataframe dfSprint zijn en een dag (als index) van een rij in dfKalenderCrew (verschil ten opzichten van 2020-1-6)
        
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempSprint = pickle.loads(pickle.dumps(dfSprint))
    dfTempPrep = pickle.loads(pickle.dumps(dfAddPrep))
    dfTempPost = pickle.loads(pickle.dumps(dfAddPost))
    dfCopyCrew = pickle.loads(pickle.dumps(dfCrew))
#     tabulist=[]               #################
    for crewlijst in list(itertools.combinations(range(0,13), int(taskdf.MinReqCrew-1))):
#         checkTabu=True             #################
#         for i in tabulist:    #################
#             if i in crewlijst:#################
#                 checkTabu=False   #################
#         if checkTabu:            ################
#             tabulist.append(crewlijst) #################
        checkUren, dfCrew = CheckCrewUren(crewlijst, dfCrew, taskdf.Duration)
        if (checkUren): # & (checkTabu):  #############
            
            a = list(set(crewlijst))
            try:
                a.remove(20)
            except:
                None

            if ((len(set(a) & set(crewA)) >= 1) | (len(crewA)==0)):
                lijstVrijeUren = [0]*16
                controle = True
                dagIndexKopie = dagIndex

                freq = 1
                while (freq <= int(taskdf.Frequency)) & (taskdf.Voltooid == False):
                    print("start van de while")        
                    if (taskdf.Eis != 'no') & (dagIndex == dagIndexKopie):
                        eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
                        lijstVrijeUren = [1]*int(eis["Moment Voltooid"]) + [0]*(16-int(eis["Moment Voltooid"]))
                    for i in crewlijst:
                        if (i == 20):
                            break
                        if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(taskdf.Duration)):
                            controle = False
                            
                            break 
                        lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
                    duration = [0]*int(round(taskdf.Duration*2))
                    print("duration wordt gezet en controle: " + str(controle))
                    if (Hulpfuncties.contains(duration, lijstVrijeUren) != False) & controle:
                        print("rij begin en eind")
                        rijbegin , rijeind = Hulpfuncties.contains(duration, lijstVrijeUren)

                        kamerGevonden = False
                        for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]:
                            if (Hulpfuncties.contains(duration, dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[1:].tolist()) != False):
                                dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1] = taskdf.Task + " freq: " + str(freq)
                                kamerGevonden = True
                                controle = True
                                break
                            elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1])& (kamerGevonden == False)):
                                controle = False
                        if (controle == True):
                            print("controle is True")
                            for i in crewlijst:
                                if i == 20:
                                    break

                                dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task + " freq: " + str(freq) # + " " + str(taskdf.Mogelijkheden.dfMogelijkheden.iloc[crewlijst,0].iloc[0])
                                dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration)
                            print("inplannen voor alle crew is gedaan")
                            if (freq == int(taskdf.Frequency)):
                                print("freq is max freq")
                                dfPrep = pd.DataFrame(dfAddPrep[dfAddPrep['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
                                dfPost = pd.DataFrame(dfAddPost[dfAddPost['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
                                print("crew toewijzen en zetten in dfSprint")
                                dfSprint.at[dfSprint.index[dfSprint['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                                geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                          prep = True)

                                geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                          prep = False)

                                if (geluktPost & geluktPrep):

                                    geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom,dfCrew=dfCrew, 
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                            prep = True, inplannen=True)
                                    geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                          dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                          dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                          prep = False, inplannen=True)

                                    dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Moment Voltooid'] = rijeind +1
                                    dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie
                                    dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Voltooid'] = True
                                    

                                    print(str(taskdf.Task) + " is ingepland, met prep en post")
                                    return [dfKalenderCrew, dfKalenderRoom, dfSprint, dfAddPrep, dfAddPost, dfCrew]
                            
                            print(str(taskdf.Task) + " freq: " + str(freq) + " crew: " +str(crewlijst) + "task voltooid: "+ str(taskdf.Voltooid))    
                            freq += 1
                            
                    else:
                        dagIndexKopie+=1
                        print("dagIndex verhogen naar: " + str(dagIndexKopie))
                        controle = True
                        lijstVrijeUren = [0]*16
                        if (dagIndexKopie == 10):
                            
                            dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                            dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                            dfSprint = pickle.loads(pickle.dumps(dfTempSprint))
                            dfAddPrep = pickle.loads(pickle.dumps(dfTempPrep))
                            dfAddPost = pickle.loads(pickle.dumps(dfTempPost))
                            dfCrew = pickle.loads(pickle.dumps(dfCopyCrew))
                            break
    return [dfTempCrew, dfTempRoom, dfTempSprint, dfTempPrep, dfTempPost, dfCopyCrew]
    
    

In [22]:
iteratie = 0
while (dfSprint.Voltooid.sum()< len(dfSprint)) & (iteratie < 1):

    print("kopieen gemaakt, iteratie is: "+str(iteratie))
    for i in range(0,len(dfSprint)):
        task = dfSprint.iloc[i,:]
        print(task.Task)
        dagIndex = 5

        crewA = []
        if task.Eis != 'no':
            eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
            dagIndex = int(eis['Dag Voltooid'])
            crewA = eis.Crew
        dfKalenderCrew, dfKalenderRoom, dfSprint, dfSprintAddprep, dfSprintAddpost, dfCrew = InplannenOpdracht(taskdf=task,
                                                               dagIndex=dagIndex, crewA=crewA,
                                                               dfKalenderCrew=dfKalenderCrew, dfCrew=dfCrew,
                                                               dfKalenderRoom=dfKalenderRoom,
                                                               dfAddPrep=dfSprintAddprep, dfAddPost=dfSprintAddpost,
                                                               dfSprint=dfSprint)
    iteratie += 1
    print(dfSprint.iloc[:,[0,20,21,22,23,27,28,29,30,31]])


kopieen gemaakt, iteratie is: 0
S1C1T1A


C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


S1C2T2A
S1C2T3A
S1C3T5A
S1C3T2A
S1C1T4
S1C1T2
S1C1T3
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 6
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 6
start van de while
duration wordt gezet en controle: True
rij begin en eind
controle is True
inplannen voor alle crew is gedaan
S1C1T3 freq: 1 crew: (1,)task voltooid: False
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: True
rij begin en eind
controle is True

dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
duration wordt gezet en controle: True
rij begin en eind
controle is True
inplannen voor alle crew is gedaan
S1C1T3 freq: 1 crew: (10,)task voltooid: False
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 6
start van de while
duration wordt gezet en controle: True
rij begin en eind
controle is True
inplannen voor alle crew is gedaan
S1C1T3 freq: 2 crew: (10,)task voltooid: False
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: True
rij begin en eind
controle is True
inplannen voor alle crew is gedaan
S1C1T3 freq: 3 crew: (10,)task voltooid: False
start van de while
duration wordt gezet en controle: True
rij begin en eind
controle is True
inplannen voor 

C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


...

C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


post S1C1T3is daadwerkelijk ingepland
prep S1C1T3is daadwerkelijk ingepland


C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


post S1C1T3is daadwerkelijk ingepland
S1C1T3 is ingepland, met prep en post
S1C1T1B
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: False
dagIn

controle is True
inplannen voor alle crew is gedaan
S1C1T1B freq: 2 crew: (9, 10)task voltooid: False
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 7
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 8
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 9
start van de while
duration wordt gezet en controle: False
dagIndex verhogen naar: 10
S1C3T3
S1C2T3B
S1C2T2B
start van de while
dur

In [20]:
(list(itertools.combinations(range(0,13), 2-1)))

[(0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (6,),
 (7,),
 (8,),
 (9,),
 (10,),
 (11,),
 (12,)]

In [24]:
dfSprint.iloc[:,[0,20,21,22,23,25,27,28,29,30,31]]

,Task,MinReqCrew,MaxReqCrew,Frequency,Duration,Crew,AantalMogelijkheden,Voltooid,Eis,Moment Voltooid,Dag Voltooid
0,S1C1T1A,2.0,6.0,4.0,2.0,"[9, 0, 20, 20, 20, 20]",8.0,True,no,13,6
1,S1C2T2A,1.0,2.0,4.0,2.0,"[11, 20, 20, 20, 20, 20]",4.0,True,no,13,6
2,S1C2T3A,1.0,2.0,2.0,1.5,"[4, 20, 20, 20, 20, 20]",2.0,True,no,3,7
3,S1C3T5A,1.0,2.0,2.0,1.5,"[5, 20, 20, 20, 20, 20]",2.0,True,no,6,5
4,S1C3T2A,1.0,6.0,1.0,2.0,"[11, 20, 20, 20, 20, 20]",1.0,True,no,4,7
5,S1C1T4,3.0,6.0,8.0,2.0,"[8, 6, 5, 20, 20, 20]",24.0,True,no,12,9
6,S1C1T2,2.0,6.0,8.0,2.0,"[11, 2, 20, 20, 20, 20]",16.0,True,no,12,9
7,S1C1T3,2.0,6.0,8.0,2.0,"(12,)",16.0,True,no,12,9
8,S1C1T1B,3.0,6.0,4.0,2.0,20,12.0,False,S1C1T1A,,
9,S1C3T3,2.0,6.0,4.0,2.0,"[7, 0, 20, 20, 20, 20]",8.0,True,no,4,9


In [47]:
a = [1,2,3,0,0,0]
b = [0,0,5]

In [48]:
a[-2:] == [0,0]

True

In [49]:
a.pop()
a.pop()

0

In [50]:
a

[1, 2, 3, 0]

In [51]:
a.extend(b)

In [52]:
a

[1, 2, 3, 0, 0, 0, 5]

In [53]:
if False:
    print(0)
else: 
    print(1)
if True:
    print(2)

1
2


In [62]:
def get_last_non_zero_index(d, default=0):
    rev = (len(d) - idx for idx, item in enumerate(d, 1) if item)
    return next(rev, default)

In [86]:
def index_of_first_nonzero(lst, duration, default= False):
    for i, value in enumerate(lst):
        if value != 0:
            return i >= duration
    return True if (sum(lst)== 0) else default

In [3]:
past = index_of_first_nonzero([0,1], 2)

NameError: name 'index_of_first_nonzero' is not defined

In [64]:
past == True

True

In [90]:
past

False

In [1]:
a = [0,1,2]


In [2]:
a[1:1]

[]

In [4]:
-6>0


False

In [2]:
2.33*3

6.99